## <span style="color: Gold"> **문장 임베딩**

- Cross-Encoder : BERT방식
    - 두 문장을 한 번에 입력해야 함  
    - 문장 쌍 1개마다 모델을 다시 실행해야 함  
    - 문장 수가 n개라면 비교 비용이 **O(n²)**  
    - 즉, 문서가 많으면 계산량이 폭발 → **검색 시스템에 부적합**

- Sentence-Transformers : SBERT 방식
    - 문장을 **하나씩 독립적으로 인코딩** 
    - BERT 의 출력을 Pooling해서 각 문장을 **고정 길이 벡터로 저장**  
    - 이후 비교는 **코사인 유사도만 계산하면 됨 → O(n)**  
    - 빠르고 확장성 좋음 → **RAG, 검색, 문서 분류 실무 표준**

- Pooling : 토큰벡터를 문장 의미 벡터로 요약하는게 풀링
    - BERT는 토큰마다 임베딩을 만들기 때문에  
    - → 이를 한 문장의 임베딩으로 변환하는 과정이 필요함  
    - → 이 과정이 **Pooling**

- Pooling 방식별 차이
    - [cls] Pooling : 첫번째 토큰([cls])만 사용 / 속도는 빠름 / 하지만 문장 의미를 완전히 반영하지 못해 **정확도 낮은 편**
    - **mean pooling(가장 중요, 실무표준)** : 모든 토큰 임베딩의 평균을 사용 / 의미 안정적, 잡음 적음 /  **SBERT에서도 기본적으로 사용 → 가장 성능 좋음**
    - max pooling : 각 차원의 최대값 사용 / 특정 특징을 강조하는 특성 / 하지만 일관성이 떨어져 **불안정한 경우 많음**

- 한국어 모델 활용 (KR-SBERT)

In [ ]:
# Sentence Transfomer
# HuggingFace에 등록된 한국어 SBERT 모델을 바로 불러옴
# 이 모델은 이미 “문장 의미 임베딩”을 잘 나오도록 학습됨
# 내부적으로는:
    # Transformer(klue-roberta-base 계열)
    # Mean Pooling
    # NLI, STS 데이터로 파인튜닝
    # 이런 과정이 이미 되어 있음.

# ✔️ SBERT에서 하는 일
# BERT가 만든 토큰 벡터들을 Pooling(mean/cls/max) 해서 → **"하나의 고정 길이 문장 벡터"**로 만든다.
# [나는] → token embedding
# [오늘] → token embedding
# [기분이] → token embedding
# [좋다] → token embedding
# ➡ SBERT에서는 이걸 Pooling해서
# 문장 임베딩(1개 벡터) → 768차원
# "Pooling해서 고정 크기 문장 벡터를 만든다"는 것은 SBERT 방식이며, Sentence-Transformers의 핵심 동작이다.
# BERT 단독으로는 문장 벡터를 만들지 않는다.


from sentence_transformers import SentenceTransformer
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
sentences = [
    '오늘 날씨가 좋아요',
    '오늘 하늘이 맑아요',
    '프로그램을 배우고 싶습니다'
]
embeddings = model.encode(sentences)  # 문장 리스트3개를 전달
print(f'임베딩 크기 : {embeddings.shape}') # 각 문장을 768차원의 벡터로 변환

# 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
sim = cosine_similarity(embeddings)
print ( f'유사도 행렬 : {sim}')  # 문장간 의미 유사도를 계산함

# 출력 유사도 행렬 해석
# 문장1 에 대한 : 문장1 - 문장2 - 문장3 간의 의미 유사도
# 문장2 에 대한 ; 문장1 - 문장2 - 문장3 간의 의미 유사도
# 문장3 에 대한 ; 문장1 - 문장2 - 문장3 간의 의미 유사도

c:\Users\playdata2\miniconda3\envs\OPENAI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


임베딩 크기 : (3, 768)
유사도 행렬 : [[0.99999976 0.80070245 0.2832657 ]
 [0.80070245 1.0000001  0.22524579]
 [0.2832657  0.22524579 0.9999999 ]]


In [ ]:
# 2. Transformer를 직접 불러와서 커스텀 SBERT 만들기

from sentence_transformers import SentenceTransformer, models
transformer = models.Transformer('klue/roberta-base')  # 이 모델은 문장-> 토큰 임베딩만 해줌 (문장 전체 임베딩은 없음)


# pooling 레이어 추가
# ✔️ 핵심 포인트: SBERT는 Pooling이 생명임
# BERT는 각 토큰마다 벡터를 만들기 때문에
# → 문장 벡터를 만들려면 Pooling 필요
# 설정 내용:
# mean pooling만 True → 전체 토큰 평균 벡터 사용
# CLS 토큰은 사용 X
# Max pooling도 사용 X
# 👉 실무에서 가장 자주 쓰는 설정
# 👉 문장 의미를 가장 안정적으로 잘 표현

pooling = models.Pooling(
    transformer.get_word_embedding_dimension(),
    pooling_mode_mean_tokens= True, # mean pooling 사용
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

# 모델 조립
model = SentenceTransformer(modules=[transformer, pooling])  # Transformer (토큰 임베딩), #Pooling (문장 임베딩)

# 문장 임베딩 생성
sentences = [
    '오늘 날씨가 좋아요',
    '오늘 하늘이 맑아요',
    '프로그램을 배우고 싶습니다'
]
embeddings= model.encode(sentences)
print(f'임베딩 크기 : {embeddings.shape}')


# 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
sim = cosine_similarity(embeddings)
print ( f'유사도 행렬 : {sim}')

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


임베딩 크기 : (3, 768)
유사도 행렬 : [[0.9999999  0.91393733 0.68440783]
 [0.91393733 0.99999976 0.7117275 ]
 [0.68440783 0.7117275  0.99999976]]
